In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import os
import json


def transform_data(data, start_idx=0):
    new_data = []
    identity_counter = start_idx

    for conversation, triples in data:
        triples_text = []
        for triple in triples:
            x, y, r = triple["x"], triple["y"], triple["r"][0] if triple["r"] else None
            if r:
                triples_text.append({
                    "x": x,
                    "r": r,
                    "y": y
                })
        
        conversation_text = "" + str(triples_text)
        output_text = "\n".join(conversation)

        conversation_entry = {
            "id": f"identity_{identity_counter}",
            "conversations": [
                {
                    "from": "human",
                    "value": output_text
                    
                },
                {
                    "from": "gpt",
                    "value": conversation_text
                }
            ]
        }
        new_data.append(conversation_entry)
        identity_counter += 1

    return new_data

In [34]:


import os
import json

# Paths
INPUT_DIR = "/home/murilo/RelNetCare/data/raw/dialog-re"
OUTPUT_DIR = "/home/murilo/RelNetCare/data/processed/dialog-re-llama"
files = ['train', 'dev']
last_data_idx = 0
new_format = []

# Check and create directory if not exists
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

for f in files:
    input_data_path = os.path.join(INPUT_DIR, f'{f}.json')
    
    # Ensure the file exists before attempting to open it
    if os.path.exists(input_data_path):
        with open(input_data_path, encoding='utf8') as fp:
            data = json.load(fp)
        
        new_format.extend(transform_data(data, start_idx=last_data_idx))
        last_data_idx = len(data)

output_data_path = os.path.join(OUTPUT_DIR, 'dialog-re-llama-train-dev.json')

# Here, we'll write to the file instead of reading it.
# Assuming you want to save the newly formatted data.
with open(output_data_path, 'w', encoding='utf8') as fp:
    json.dump(new_format, fp)
